In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import io
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_datasets.core import DatasetInfo
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.models import Sequential
print(tf.__version__)

2.3.0


In [8]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [11]:
train_data: tf.raw_ops.PrefetchDataset = imdb["train"]
test_data: tf.raw_ops.PrefetchDataset = imdb["test"]

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [17]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(
    num_words=vocab_size, 
    oov_token=oov_tok
)
tokenizer.fit_on_texts(training_sentences)
word_index: dict = tokenizer.word_index

sequences: [[int]] = tokenizer.texts_to_sequences(training_sentences)
padded: np.ndarray = pad_sequences(
    sequences, 
    maxlen=max_length, 
    truncating=trunc_type
)

testing_sequences: [[int]] = tokenizer.texts_to_sequences(testing_sentences)
testing_padded: np.ndarray = pad_sequences(
    testing_sequences,
    maxlen=max_length
)

In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [26]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding_1", input_length=max_length),
    Flatten(name="flatten_2"),
    Dense(units=6, activation='relu', name="dense_2"),
    Dense(units=1, activation='sigmoid', name="dense_3")
], name="IMDB_classfier")
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "IMDB_classfier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 10
model.fit(
    x=padded, 
    y=training_labels_final, 
    validation_data=(testing_padded, testing_labels_final),
    epochs=num_epochs, 
    workers=-1,
    use_multiprocessing=True
)

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4980 - accuracy: 0.7379 - val_loss: 0.3534 - val_accuracy: 0.8448
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2415 - accuracy: 0.9070 - val_loss: 0.3695 - val_accuracy: 0.8384
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0969 - accuracy: 0.9745 - val_loss: 0.4350 - val_accuracy: 0.8310
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0241 - accuracy: 0.9970 - val_loss: 0.5250 - val_accuracy: 0.8277
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0063 - accuracy: 0.9995 - val_loss: 0.6043 - val_accuracy: 0.8227
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0025 - accuracy: 0.9998 - val_loss: 0.6408 - val_accuracy: 0.8270
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.5872e-04 - accuracy: 1.0000 - val_loss: 0.6792 - val_accuracy: 0.8293
Ep

In [28]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [30]:
out_v = io.open('../data/ImdbReviews/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('../data/ImdbReviews/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

<b>把"../data/ImdbReviews/vecs.tsv"文件和"../data/ImdbReviews/meta.tsv"文件放到<a href="https://projector.tensorflow.org/" target="_blank">这个网址</a>进行可视化展示</b>

In [31]:
try:
    from google.colab import files
except ImportError as e:
    print(e)
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

No module named 'termios'


In [33]:
sentence = "I really think this is amazing. honest."
sequence: [[int]] = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
